## Notebook d'administration : groupes et vues par secteur géographique de l'enseigne Jardi

#### Connexion au gis, import des bibliothèques et gestion des avertissements

In [ ]:
import logging
logging.captureWarnings(True)

from arcgis.gis import GIS
gis = GIS("home")
import pandas as pd
from arcgis.features import FeatureLayerCollection

#### Lecture du Feature Layer

In [ ]:
# Item Added From Toolbar
# Title: Enseignes_Jardi_2022 | Type: Feature Service | Owner: plouis_esrifrance
enseignes = gis.content.get("777283a81da148f5a2a519ef98fb4502")
enseignes

In [ ]:
df = pd.DataFrame.spatial.from_layer(enseignes.layers[0])
df

In [ ]:
dict = {}
for i in df.index:
    dict[df["secteur_com"][i]]=df["responsable_secteur"][i]
dict

#### Création des groupes et ajout des utilisateurs

In [ ]:
for secteur,respo in dict.items() : 
    groupe = gis.groups.create(title='Secteur {0}'.format(secteur),
                                    tags = [secteur,'Jardi, magasins'],
                                    description = 'Magasins Jardi pour le secteur {0}'.format(secteur),
                                    snippet = 'Vues des magasins Jardi pour le secteur {0}'.format(secteur),
                                    access = 'private',
                                    is_invitation_only = 'True',
                                    thumbnail = r'/arcgis/home/Jardi{0}.png'.format(secteur)
                              )
    display(groupe)
    utilisateur = gis.users.search(respo)[0]
    display(utilisateur)
    groupe.add_users([utilisateur])

#### Création des vues

In [ ]:
source_search = gis.content.search("777283a81da148f5a2a519ef98fb4502")[0] 
source_flc  = FeatureLayerCollection.fromitem(source_search)

In [ ]:
for secteur in dict.keys() : 
    vue = source_flc.manager.create_view(name='Vue Magasins Jardi {0}'.format(secteur))
    view_search  = gis.content.search('Vue Magasins Jardi {0}'.format(secteur))[0]
    view_flc = FeatureLayerCollection.fromitem(view_search)
    view_layer = view_flc.layers[0]
    view_def = {"viewDefinitionQuery" : "secteur_com = '{0}'".format(secteur)}
    view_layer.manager.update_definition(view_def)
    print ('vue creee {0}'.format(secteur))

#### Création des WebMaps et Dashboards associés

In [ ]:
from arcgis.mapping import WebMap
from arcgis.apps.dashboard import Dashboard
from arcgis.apps.dashboard import Header, RichText, Indicator, MapLegend, SidePanel
from arcgis.apps.dashboard import add_row, add_column

In [ ]:
#WEBMAPS
for secteur in dict.keys() :
    view_search  = gis.content.search('Vue Magasins Jardi {0}'.format(secteur))[0]
    carte = gis.map("France",5)
    carte.add_layer(view_search)
    carte.zoom_to_layer(view_search)
    webmap_properties = {'title':'WebMap Magasins Jardi {0}'.format(secteur),
        'snippet': 'WebMap des magasins de Jardi pour le secteur {0}'.format(secteur),
        'tags':['Jardi','ArcGIS Notebooks', 'Magasin', 'Demo','SIG2022']}
    carte.save(webmap_properties)

In [ ]:
#DASHBOARDS
for secteur in dict.keys() :
    
    #webmap à intégrer
    carte_search = gis.content.search('WebMap Magasins Jardi {0}'.format(secteur))[0]
    carteid = carte_search.id
    carte_get = gis.content.get("{0}".format(carteid))
    webmap = WebMap(carte_get) 
    webmap.height=1
    webmap.basemap_switcher = True
    webmap.search = True
    
    #configuration des popups
    webmap.pop_ups=True  
    layer = webmap.layers[0]
    layer.popupInfo.fieldInfos[0].visible = False
    layer.popupInfo.fieldInfos[1].visible = False
    layer.popupInfo.fieldInfos[2].visible = False
    layer.popupInfo.fieldInfos[3].visible = False
    layer.popupInfo.fieldInfos[8].visible = False
    webmap.update()
    
    #entête
    entete = Header(title='Dashboard Magasins Jardi secteur {0}'.format(secteur),
                subtitle='Suivi du CA pour les années 2021 et 2022',
                margin=True,
                size='medium')
    
    #légende
    legende = MapLegend(webmap)
    legende.height=0.25
    
    #couche pour les indicateurs
    view_search  = gis.content.search('Vue Magasins Jardi {0}'.format(secteur))[0]
    view_get = gis.content.get(view_search.id)
    
    #indicateurs : CA moyens selon les années
    indicator_2020 = Indicator(item = view_get, name="CA_Total_2020", title = "CA total des magasins du secteur en 2020 en €" )
    indicator_2020.data.value_type = "statistic"
    indicator_2020.data.statistic = "sum"
    indicator_2020.data.value_field = "CA2020"
    indicator_2020.height=0.35

    indicator_2021 = Indicator(item = view_get, name="CA_Total_2021", title = "CA total des magasins du secteur en 2021 en €" )
    indicator_2021.data.value_type = "statistic"
    indicator_2021.data.statistic = "sum"
    indicator_2021.data.value_field = "CA2021"
    indicator_2021.height=0.4
    
    #création et sauvegarde du Dashboard
    dashboard = Dashboard()
    dashboard.header = entete
    dashboard.theme = "dark"
    dashboard.layout = add_column([add_row([webmap]),add_row([indicator_2020,indicator_2021,legende])])
    dashboard.save('Dashboard Magasins Jardi secteur {0}'.format(secteur),
               description="Dashboards des magasins de l'enseigne Jardi pour le secteur {0} indiquant les CA moyens pour les années 2020,2021,2022".format(secteur),
               tags='Jardi, Demo, SIG2022', 
               overwrite=True)

#### Partage des Vues, WebMaps et des Dashboards dans les groupes

In [ ]:
for secteur in dict.keys() : 
    
    #view_search  = gis.content.search('Vue Magasins Jardi {0}'.format(secteur))[0]
    #viewid = view_search.id
    #view_share  = gis.content.get("{0}".format(viewid))
    view_share  = gis.content.get(gis.content.search('Vue Magasins Jardi {0}'.format(secteur))[0].id)
    display(view_search)
    
    wm_share  = gis.content.get(gis.content.search('WebMap Magasins Jardi {0}'.format(secteur))[0].id)
    display(wm_share)

    dashboard_share  = gis.content.get(gis.content.search('Dashboard Magasins Jardi secteur {0}'.format(secteur))[0].id)
    display(wm_share)
    
    groupe = gis.groups.search('title:Secteur {0}'.format(secteur), max_groups=1)
    display(groupe)
    groupeid = groupe[0].id              
    view_share.share(groups=[groupeid])
    wm_share.share(groups=[groupeid])
    dashboard_share.share(groups=[groupeid])